# 882 Replication Paper: Flabbi 2010

- Read in data
- Define log-normal pdf & cdf 
- Estimate men and women separately to allow for parameters to very by gender

## Import Packages

In [1]:
# Data Manipulation 
import numpy as np
import pandas as pd

# General
import pdb

# Estimation
from scipy.optimize import minimize
import scipy.stats as stats
# import numdifftools as ndt

# Data Visualization
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn.apionly as sns
from pylab import *

/Users/megan.mccoy/anaconda3/lib/python3.6/site-packages/seaborn/apionly.py:6: UserWarning: As seaborn no longer sets a default style on import, the seaborn.apionly module is deprecated. It will be removed in a future version.
  warnings.warn(msg, UserWarning)


## Import Data 

- CPS data on gender, wages, duration of unemployment
- M: males
- F: females
- U: unemployed
- E: employed 
- {M,F} X {E,U} = {males,females} X {employed, unemployed}

In [2]:
data=pd.read_csv('../data/est_c.csv')
data.columns = ['dur', 'wage', 'empl', 'women']

M = data[data['women']==0] #1186 men 
F = data[data['women']==1] #993 women 
U = data[data['empl']==0] #45 unemployed
E = data[data['empl']==1] #2134 employed

ME = M[M['empl']==1] #1109 employed men
MU = M[M['empl']==0] #18 unemployed men
FE = F[F['empl']==1] #966 employed women
FU = F[F['empl']==0] #27 unemployed women

In [ ]:
data.describe()

### Summary Statistics, without trimming

In [ ]:
agg_dict = {
    'wage': ['mean', 'std'],
    'dur': ['mean', 'std'],
}

In [ ]:
print(data.groupby(['women', 'empl']).agg(agg_dict).to_latex()) # by gender

In [ ]:
print(data.groupby(['empl']).agg(agg_dict).to_latex()) # all

### Summary Statistics, with trimming

In [ ]:
print(np.percentile(ME['wage'], 5))

In [ ]:
print(np.percentile(FE['wage'], 5))

In [3]:
M_WAGE_COND = (data['wage'] > 7.27) #hard coded percentile so it does not continually update
M_COND = (data['women'] == 0)

F_WAGE_COND = (data['wage'] > 5.75625) #hard coded percentile so it does not continually update
F_COND = (data['women'] == 1)

DUR_COND = (data['dur'] > 0)

trim = data[ (M_COND & M_WAGE_COND) | (F_COND & F_WAGE_COND) | DUR_COND ]

# trim = data[ (M_COND & (M_WAGE_COND | DUR_COND)) | (F_COND & (F_WAGE_COND | DUR_COND)) ]

M = trim[trim['women']==0] #1071 ?? men 
F = trim[trim['women']==1] #944 ?? women 
U = trim[trim['empl']==0] #45 unemployed
E = trim[trim['empl']==1] #1970 ?? employed

ME = M[M['empl']==1] #1053 ?? employed men
MU = M[M['empl']==0] #18 unemployed men
FE = F[F['empl']==1] #917 employed women
FU = F[F['empl']==0] #27 unemployed women

In [ ]:
print(trim.groupby(['women', 'empl']).agg(agg_dict).to_latex()) # by gender

In [ ]:
print(trim.groupby(['empl']).agg(agg_dict).to_latex()) # all

## Figures 

- Distribution of wages, men and women

In [ ]:
# Seaborn
fig, ax = plt.subplots(2, 1, figsize=(12, 8))

sns.distplot(ME['wage'], color='#4B9CD3', hist_kws={'alpha' : .3}, bins=50, ax=ax[0])
sns.distplot(FE['wage'], color='#4B9CD3', hist_kws={'alpha' : .3}, bins=50, ax=ax[1])

ax[0].legend(['Men'])
ax[1].legend(['Women']) 

plt.tight_layout()

In [ ]:
fig.savefig('./figures/fig1_2.png', bbox_inches='tight', transparent=True)

## Estimation 

- Estimations 1-3: Same arrival and termination rates
- Estimations 4-6: Gender specific arrival and termination rates

In [ ]:
# Initial Conditions

wstarM = min(ME['wage'])
wstarF = min(FE['wage'])

λ_M = 0.22
λ_F = 0.22
η_M = 0.005
η_F = 0.005
# α = 0.5  #FIXED
μ_M = 3.456
μ_F= 3.454
σ_M = 0.558
σ_F = 0.423
p = 0.5
d_F = 10
# d_M = 0   #FIXED

### Wage Function

In [ ]:
def f_w(wage: np.array, α: float, wstar: float):
    """
    Wage functions
    """
    return (wage - (1-α)*wstar)*(1/α)

### Estimation 1: Same arrival and termination, Productivity Differences, No Prejudice

In [ ]:
def loglik1(params: list):
    """
    Calculates the log likelihood with the log normal distribution
    """
    
    λ = np.exp(params[0])
    η = np.exp(params[1])
    μ_M = params[2]
    σ_M = np.exp(params[3])
    μ_F = params[4]
    σ_F = np.exp(params[5])
    d_F = 0
    p = 0
    α = 0.5

    
    hM = λ * ( (1-stats.lognorm.cdf(wstarM,μ_M,σ_M)) );
    L00 = (η/(η+hM)) * hM * np.exp(-hM*np.sum(MU.values[:,0])); #unemployed men

    L01a = ( ( (1/α)*stats.lognorm.pdf(f_w(ME.values[:,1],α,wstarM),μ_M,σ_M) )/(1-stats.lognorm.cdf(wstarM,μ_M,σ_M)) );
    L01 = np.log(hM/(η+hM)) + np.sum( np.log(L01a) ); #employed men
    
    hF = λ*( (1-p)*(1-stats.lognorm.cdf(wstarF,μ_F,σ_F)) + p*(1-stats.lognorm.cdf((wstarF+d_F),μ_F,σ_F)) );
    L10 = ( (η/(η+hF)) * hF * np.exp(-hF*np.sum(FU.values[:,0])) ); #unemployed women

    L11a = ( ( ((1-p)/α)*stats.lognorm.pdf(f_w(FE.values[:,1],α,wstarF),μ_F,σ_F) ) / (1-stats.lognorm.cdf(wstarF,μ_F,σ_F)) );
    L11b = ( ( (p/α)*stats.lognorm.pdf(f_w((FE.values[:,1]+α*d_F),α,wstarF),μ_F,σ_F) ) / (1-stats.lognorm.cdf((wstarF+d_F),μ_F,σ_F)) );
    L11 = np.log(hF/(η+hF)) + np.sum( np.log(L11a + L11b) ); #employed women

    results = np.log(L00) + L01 + np.log(L10) + L11

    return -1*results

In [ ]:
init1 = [λ_M, η_M, μ_M, σ_M, μ_F, σ_F]

est1 = minimize(loglik1, init1, method='nelder-mead', options={'maxiter': 10000})

In [ ]:
# est1 default

print(est1.x)
print(-loglik1(est1.x))

In [ ]:
# est1 nelder

print(est1.x)
print(-loglik1(est1.x))

#### Test Statistics

In [ ]:
# Standard Errors

vcv_mle1 = est1.hess_inv
se1 = np.sqrt(np.diag(vcv_mle1))
print('Standard errors are given in ', se1)

# Likelihood Ratio Test

LR_val_1 = 2 * (-loglik1(est1.x) + loglik1(init1))
pval_h0_1 = 1.0 - stats.chi2.cdf(LR_val_1, 5)
print('LR value = ', LR_val_1, '. Chi squared of H0 with 5 degrees of freedom p-value = ', pval_h0_1)

### Estimation 2: Same arrival and termination, Prejudice, No Productivity Differences

In [ ]:
def loglik2(params: list):
    """
    Calculates the log likelihood with the log normal distribution
    """
  
    λ = np.exp(params[0])
    η = np.exp(params[1])
    μ = params[2]
    σ = np.exp(params[3])
    d_F = np.exp(params[5])
    p = np.exp(params[4]) / (1 + np.exp(params[4]))
    α = 0.5

#    pdb.set_trace()
    
    hM = λ * ( (1-stats.lognorm.cdf(wstarM,μ,σ)) );
    L00 = (η/(η+hM)) * hM * np.exp(-hM*np.sum(MU.values[:,0])); #unemployed men

    L01a = ( ( (1/α)*stats.lognorm.pdf(f_w(ME.values[:,1],α,wstarM),μ,σ) )/(1-stats.lognorm.cdf(wstarM,μ,σ)) );
    L01 = np.log(hM/(η+hM)) + np.sum( np.log(1+L01a) ); #employed men
    
    hF = λ * ( (1-p)*(1-stats.lognorm.cdf(wstarF,μ,σ)) + p*(1-stats.lognorm.cdf((wstarF+d_F),μ,σ)) );
    L10 = ( (η/(η+hF)) * hF * np.exp(-hF*np.sum(FU.values[:,0])) ); #unemployed women

    L11a = ( ( ((1-p)/α)*stats.lognorm.pdf(f_w(FE.values[:,1],α,wstarF),μ,σ) ) / (1-stats.lognorm.cdf(wstarF,μ,σ)) );
    L11b = ( ( (p/α)*stats.lognorm.pdf(f_w((FE.values[:,1]+ α*d_F),α,wstarF),μ,σ) ) / (1-stats.lognorm.cdf((wstarF+d_F),μ,σ)) );
    L11 = np.log(hF/(η+hF)) + np.sum( np.log(1+L11a + L11b) ); #employed women

    results = np.log(1+L00) + L01 + np.log(1+L10) + L11

    return -1*results

In [ ]:
init2 = [λ_M, η_M, μ_M, σ_M, p, d_F]

est2 = minimize(loglik2, init2, method='nelder-mead', options={'maxiter': 10000})

In [ ]:
# est2

print(est2.x)
print(-loglik2(est2.x))

#### Test Statistics

In [ ]:
# Standard Errors

vcv_mle2 = est2.hess_inv
se2 = np.sqrt(np.diag(vcv_mle2))
print('Standard errors are given in ', se2)

# Likelihood Ratio Test

LR_val_2 = 2 * (-loglik2(est2.x) + loglik2(init2))
pval_h0_2 = 1.0 - stats.chi2.cdf(LR_val_2, 5)
print('LR value = ', LR_val_2, '. Chi squared of H0 with 5 degrees of freedom p-value = ', pval_h0_2)


### Estimation 3: Same arrival and termination, Productivity Differences and Prejudice

In [ ]:
def loglik3(params: list):
    """
    Calculates the log likelihood with the log normal distribution
    """
    
    λ = np.exp(params[0])
    η = np.exp(params[1])
    μ_M = params[2]
    σ_M = np.exp(params[3])
    μ_F = params[4]
    σ_F = np.exp(params[5])
    d_F = np.exp(params[7])
    p = (np.exp(params[6]))/(1+np.exp(params[6]))
    α = 0.5
    
#    pdb.set_trace()
    
    hM = λ * ( (1-stats.lognorm.cdf(wstarM,μ_M,σ_M)) );
    L00 = (η/(η+hM)) * hM * np.exp(-hM*np.sum(MU.values[:,0])); #unemployed men

    L01a = ( ( (1/α)*stats.lognorm.pdf(f_w(ME.values[:,1],α,wstarM),μ_M,σ_M) )/(1-stats.lognorm.cdf(wstarM,μ_M,σ_M)) );
    L01 = np.log(hM/(η+hM)) + np.sum( np.log(L01a) ); #employed men
    
    hF = λ * ( (1-p)*(1-stats.lognorm.cdf(wstarF,μ_F,σ_F)) + p*(1-stats.lognorm.cdf((wstarF+d_F),μ_F,σ_F)) );
    L10 = ( (η/(η+hF)) * hF * np.exp(-hF*np.sum(FU.values[:,0])) ); #unemployed women

    L11a = ( ( ((1-p)/α)*stats.lognorm.pdf(f_w(FE.values[:,1],α,wstarF),μ_F,σ_F) ) / (1-stats.lognorm.cdf(wstarF,μ_F,σ_F)) );
    L11b = ( ( (p/α)*stats.lognorm.pdf(f_w((FE.values[:,1]+α*d_F),α,wstarF),μ_F,σ_F) ) / (1-stats.lognorm.cdf((wstarF+d_F),μ_F,σ_F)) );
    L11 = np.log(hF/(η+hF)) + np.sum( np.log(L11a + L11b) ); #employed women

    results = np.log(L00) + L01 + np.log(L10) + L11

    return -1*results

In [ ]:
init3 = [λ_M, η_M, μ_M, σ_M, μ_F, σ_F, p, d_F]

est3 = minimize(loglik3, init3, options={'maxiter': 10000})

In [ ]:
print(est3.x)
print(-loglik3(est3.x))

#### Test Statistics

In [ ]:
# Standard Errors

vcv_mle3 = est3.hess_inv
se3 = np.sqrt(np.diag(vcv_mle3))
print('Standard errors are given in ', se3)

# Likelihood Ratio Test

LR_val_3 = 2 * (-loglik3(est3.x) + loglik3(init3))
pval_h0_3 = 1.0 - stats.chi2.cdf(LR_val_3, 5)
print('LR value = ', LR_val_3, '. Chi squared of H0 with 5 degrees of freedom p-value = ', pval_h0_3)


### Estimation 4: Productivity Differences, No Prejudice

In [ ]:
def loglik4(params: list):
    """
    Calculates the log likelihood with the log normal distribution
    """
    
    λ_M = np.exp(params[0])
    λ_F = np.exp(params[1])
    η_M = np.exp(params[2])
    η_F = np.exp(params[3])
    μ_M = params[4]
    σ_M = np.exp(params[5])
    μ_F = params[6]
    σ_F = np.exp(params[7])
    d_F = 0
    p = 0
    α = 0.5
    
    hM = λ_M * ( (1-stats.lognorm.cdf(wstarM,μ_M,σ_M)) );
    L00 = (η_M/(η_M+hM)) * hM * np.exp(-hM*np.sum(MU.values[:,0])); #unemployed men

    L01a = ( ( (1/α)*stats.lognorm.pdf(f_w(ME.values[:,1],α,wstarM),μ_M,σ_M) )/(1-stats.lognorm.cdf(wstarM,μ_M,σ_M)) );
    L01 = np.log(hM/(η_M+hM)) + np.sum( np.log(L01a) ); #employed men
    
    hF = λ_F*( (1-p)*(1-stats.lognorm.cdf(wstarF,μ_F,σ_F)) + p*(1-stats.lognorm.cdf((wstarF+d_F),μ_F,σ_F)) );
    L10 = ( (η_F/(η_F+hF)) * hF * np.exp(-hF*np.sum(FU.values[:,0])) ); #unemployed women

    L11a = ( ( ((1-p)/α)*stats.lognorm.pdf(f_w(FE.values[:,1],α,wstarF),μ_F,σ_F) ) / (1-stats.lognorm.cdf(wstarF,μ_F,σ_F)) );
    L11b = ( ( (p/α)*stats.lognorm.pdf(f_w((FE.values[:,1]+α*d_F),α,wstarF),μ_F,σ_F) ) / (1-stats.lognorm.cdf((wstarF+d_F),μ_F,σ_F)) );
    L11 = np.log(hF/(η_F+hF)) + np.sum( np.log(L11a + L11b) ); #employed women

    results = np.log(L00) + L01 + np.log(L10) + L11

    return -1*results

In [ ]:
init4 = [λ_M, λ_F, η_M, η_F, μ_M, σ_M, μ_F, σ_F]

est4 = minimize(loglik4, init4, method='nelder-mead', options={'maxiter': 10000})

In [ ]:
print(est4.x)
print(-loglik4(est4.x))

#### Test Statistics

In [ ]:
# Standard Errors

vcv_mle4 = est4.hess_inv
se4 = np.sqrt(np.diag(vcv_mle4))
print('Standard errors are given in ', se4)

# Likelihood Ratio Test

LR_val_4 = 2 * (-loglik4(est4.x) + loglik4(init4))
pval_h0_4 = 1.0 - stats.chi2.cdf(LR_val_4, 7)
print('LR value = ', LR_val_4, '. Chi squared of H0 with 7 degrees of freedom p-value = ', pval_h0_4)


### Estimation 5: Prejudice, No Productivity Differences

In [ ]:
def loglik5(params: list):
    """
    Calculates the log likelihood with the log normal distribution
    """
    
    λ_M = np.exp(params[0])
    λ_F = np.exp(params[1])
    η_M = np.exp(params[2])
    η_F = np.exp(params[3])
    μ = params[4]
    σ = np.exp(params[5])
    d_F = np.exp(params[7])
    p = (np.exp(params[6]))/(1+np.exp(params[6]))
    α = 0.5
    
#    pdb.set_trace()
    
    hM = λ_M * ( (1-stats.lognorm.cdf(wstarM,μ,σ)) );
    L00 = (η_M/(η_M+hM)) * hM * np.exp(-hM*np.sum(MU.values[:,0])); #unemployed men

    L01a = ( ( (1/α)*stats.lognorm.pdf(f_w(ME.values[:,1],α,wstarM),μ,σ) )/(1-stats.lognorm.cdf(wstarM,μ,σ)) );
    L01 = np.log(hM/(η_M+hM)) + np.sum( np.log(1 + L01a) ); #employed men
    
    hF = λ_F*( (1-p)*(1-stats.lognorm.cdf(wstarF,μ,σ)) + p*(1-stats.lognorm.cdf((wstarF+d_F),μ,σ)) );
    L10 = ( (η_F/(η_F+hF)) * hF * np.exp(-hF*np.sum(FU.values[:,0])) ); #unemployed women

    L11a = ( ( ((1-p)/α)*stats.lognorm.pdf(f_w(FE.values[:,1],α,wstarF),μ,σ) ) / (1-stats.lognorm.cdf(wstarF,μ,σ)) );
    L11b = ( ( (p/α)*stats.lognorm.pdf(f_w((FE.values[:,1]+α*d_F),α,wstarF),μ,σ) ) / (1-stats.lognorm.cdf((wstarF+d_F),μ,σ)) );
    L11 = np.log(hF/(η_F+hF)) + np.sum( np.log(1 + L11a + L11b) ); #employed women

    results = np.log(1+L00) + L01 + np.log(1+L10) + L11

    return -1*results

In [ ]:
init5 = [λ_M, λ_F, η_M, η_F, μ_M, σ_M, p, d_F]

est5 = minimize(loglik5, init5, method='nelder-mead', options={'maxiter': 10000})

In [ ]:
est5.x

### Estimation 6: Productivity Differences and Prejudice

In [ ]:
def loglik6(params: list):
    """
    Calculates the log likelihood with the log normal distribution
    """
    
    λ_M = np.exp(params[0])
    λ_F = np.exp(params[1])
    η_M = np.exp(params[2])
    η_F = np.exp(params[3])
    μ_M = params[4]
    σ_M = np.exp(params[5])
    μ_F = params[6]
    σ_F = np.exp(params[7])
    d_F = np.exp(params[9])
    p = (np.exp(params[8]))/(1+np.exp(params[8]))
    α = 0.5
    
#    pdb.set_trace()
    
    hM = λ_M * ( (1-stats.lognorm.cdf(wstarM,μ_M,σ_M)) );
    L00 = (η_M/(η_M+hM)) * hM * np.exp(-hM*np.sum(MU.values[:,0])); #unemployed men

    L01a = ( ( (1/α)*stats.lognorm.pdf(f_w(ME.values[:,1],α,wstarM),μ_M,σ_M) )/(1-stats.lognorm.cdf(wstarM,μ_M,σ_M)) );
    L01 = np.log(hM/(η_M+hM)) + np.sum( np.log(1 + L01a) ); #employed men
    
    hF = λ_F*( (1-p)*(1-stats.lognorm.cdf(wstarF,μ_F,σ_F)) + p*(1-stats.lognorm.cdf((wstarF+d_F),μ_F,σ_F)) );
    L10 = ( (η_F/(η_F+hF)) * hF * np.exp(-hF*np.sum(FU.values[:,0])) ); #unemployed women

    L11a = ( ( ((1-p)/α)*stats.lognorm.pdf(f_w(FE.values[:,1],α,wstarF),μ_F,σ_F) ) / (1-stats.lognorm.cdf(wstarF,μ_F,σ_F)) );
    L11b = ( ( (p/α)*stats.lognorm.pdf(f_w((FE.values[:,1]+α*d_F),α,wstarF),μ_F,σ_F) ) / (1-stats.lognorm.cdf((wstarF+d_F),μ_F,σ_F)) );
    L11 = np.log(hF/(η_F+hF)) + np.sum( np.log(1 + L11a + L11b) ); #employed women

    results = np.log(1+L00) + L01 + np.log(1+L10) + L11

    return -1*results

In [ ]:
init6 = [λ_M, λ_F, η_M, η_F, μ_M, σ_M, μ_F, σ_F, p, d_F]

est6 = minimize(loglik6, init6)#options={'maxiter': 10000})

In [ ]:
est6.x

In [ ]:
pd.DataFrame(est1.x,est2.x,est3.x,est4.x,est5.x,est6.x)

## Scratch

### Initial Conditions

In [49]:
λ = 0.22
λ_M = 0.18
λ_F = 0.28
η = 0.005
η_M = 0.003
η_F = 0.0077
μ_M = 3.456
μ_F= 3.454
μ = 3.433
σ_M = 0.558
σ_F = 0.423
σ = 0.523
p = 0.5
d = 15

### Parameters without distributional assumptions

In [27]:
wstarM = min(ME['wage'])
wstarF = min(FE['wage'])

hM = MU['dur'].count()/sum(MU.values[:,0])
hF = FU['dur'].count()/sum(FU.values[:,0])
h = U['dur'].count()/sum(U.values[:,0])

ηM = hM * (MU['dur'].count()/ME['empl'].count())
ηF = hF * (FU['dur'].count()/FE['empl'].count())
η = h * (U['dur'].count()/E['empl'].count())

α = 0.5

### Accepted Wage Function and Distributions

In [28]:
def dens_accepted(wage: np.array, α: float, μ: float, σ: float, wstar: float):
    """
    Calculates the density of accepted wages
    """
    
    l = (α*μ) + ((1-α)*wstar)
    s = α * σ
    shape = 1
    
    sf_in = (wstar - l)/s
    
    return stats.lognorm.pdf(wage, shape, l, s) / stats.lognorm.sf(sf_in, shape, l, s)

In [29]:
def dens_accepted_prej(wage: np.array, α: float, μ: float, σ: float, wstar: float, d: float):
    """
    Calculates the density of accepted wages when prejudice is present
    """
    
    l = (α*μ) + ((1-α)*wstar) - α*d
    s = α * σ
    shape = 1
    
    sf_in = (wstar - l)/s
    
    return stats.lognorm.pdf(wage,shape,l,s) / stats.lognorm.sf(sf_in, shape, l, s)

In [32]:
def lambdaM(h: float, wstarM: float, α: float, μ: float, σ: float):
    """
    Estimates lambda for men
    """
    
    l = (α*μ) + ((1-α)*wstarM)
    s = α * σ
    shape = 1
    
    sf_in = (wstarM-l)/s
    
    denom = stats.lognorm.sf(sf_in, shape, l, s)
    
    return h/denom

In [33]:
def lambdaF(h: float, wstarF: float, α: float, μ: float, σ: float, p: float):
    """
    Estimates lambda for women
    """
    
    l1 = (α*μ) + ((1-α)*wstarF)
    l2 = (α*μ) + ((1-α)*wstarF) - α*d
    s = α * σ
    shape = 1
    
    sf_in1 = (wstarF-l1)/s
    sf_in2 = (wstarF-l2)/s
    
    denom = (1-p)*stats.lognorm.sf(sf_in1, shape, l1, s) + p*stats.lognorm.sf(sf_in2, shape, l2, s)
    
    return h/denom

In [ ]:
# def hM(λM: float, wstarM: float, α: float, μ: float, σ: float):
    
#     l = (α*μ) + ((1-α)*wstarM)
#     s = α * σ
#     shape = .5
    
#     return λM * stats.lognorm.sf(wstarM,shape, l, s)

In [ ]:
# def hF(λF: float, wstarF: float, α: float, μ: float, σ: float, p: float):
    
#     l1 = (α*μ) + ((1-α)*wstarF)
#     l2 = (α*μ) + ((1-α)*wstarF) - α*d
#     s = α * σ
#     shape = .5
    
#     mult = (1-p)*stats.lognorm.sf(wstarF, shape, l1, s) + p*stats.lognorm.sf(wstarF, shape, l2, s)
    
#     return λF * mult

### Likelihood Functions

#### Estimation 6: Working!

In [34]:
def loglik_6( params: list ):
    """
    Calculates log likelihood with prejudice and productivity differences 
    
    Estimation 6 (ηM, ηF, hM, hF)
    
    Parameters to estimate: 
        λM # see fxn elsewhere
        λF # see fxn elsewhere
        ηM ?
        ηF ?
        μM
        σM
        μF
        σF
        d
        p
    """
    
    λM = np.exp(params[0])
    λF = np.exp(params[1])
    ηM = np.exp(params[2])
    ηF = np.exp(params[3])
    μM = params[4]
    σM = np.exp(params[5])
    μF = params[6]
    σF = np.exp(params[7])
    d = np.exp(params[8])
    p = np.exp(params[9])/(1+np.exp(params[9]))

    
    # Men's equations 
    λM = lambdaM(hM, wstarM, α, μM, σM)
    
    a = M['dur'].count() * np.log(hM/(hM+ηM))
    b = MU['dur'].count() * np.log(ηM)
    c = - hM * np.sum(MU.values[:,0])
    e = np.sum( np.log( (1/α) * dens_accepted(ME['wage'], α, μM, σM, wstarM) ) )
    
    # Women's equations
    λF = lambdaF(hF, wstarF, α, μM, σM, p)
    
    f = F['dur'].count() * np.log(hF/(hF+ηF))
    g = FU['dur'].count() * np.log(ηF)
    h = - hF * np.sum(FU.values[:,0])
    
    y = ((1-p)/α) * dens_accepted(FE['wage'], α, μF, σF, wstarF)
    z = (p/α) * dens_accepted_prej(FE['wage'], α, μF, σF, wstarF, d)

    i = np.sum( np.log( y + z ) )
    
    return (a + b + c + e + f + g + h + i)

In [35]:
# Check on log-likelihood

param6 = [λ_M, λ_F, η_M, η_F, μ_M, σ_M, μ_F, σ_F, d, p]

b6_0 = np.log(param6[0])
b6_1 = np.log(param6[1])
b6_2 = np.log(param6[2])
b6_3 = np.log(param6[3])
b6_4 = param6[4]
b6_5 = np.log(param6[5])
b6_6 = param6[6]
b6_7 = np.log(param6[7])
b6_8 = np.log(param6[8])
b6_9 = np.log(1)

init6 = [b6_0, b6_1, b6_2, b6_3, b6_4, b6_5, b6_6, b6_7, b6_8, b6_9]

print(loglik_6(init6))

-5947.5045035078465


In [36]:
est_6 = minimize(loglik_6, init6)

est_6

/Users/megan.mccoy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:38: RuntimeWarning: divide by zero encountered in log
/Users/megan.mccoy/anaconda3/lib/python3.6/site-packages/scipy/optimize/optimize.py:663: RuntimeWarning: invalid value encountered in double_scalars
  grad[k] = (f(*((xk + d,) + args)) - f0) / d[k]
/Users/megan.mccoy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:50: RuntimeWarning: divide by zero encountered in log
/Users/megan.mccoy/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1818: RuntimeWarning: overflow encountered in double_scalars
  x = np.asarray((x - loc)/scale, dtype=dtyp)
/Users/megan.mccoy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: divide by zero encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.
/Users/megan.mccoy/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1652: RuntimeWarning: divide by zero encoun

      fun: -14520.22960439797
 hess_inv: array([[ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  1.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  1.00000010e+00,
        -5.90133621e-09, -4.83535142e-05,  1.22118105e-05,
        -6.14910381e-06,  2.05661525e-05,  7.30152026e-05,
         7.39743797e-08],
       [ 0.00000000e+00,  0.00000000e+00, -5.90133621e-09,
         1.00000000e+00,  2.85890129e-06, -7.31411406e-07,
         3.65433030e-07, -1.15776735e-06, -4.34009810e-06,
        -1.01803525e-08],
       [ 0.00000000e+00,  0.00000000e+00, -4.83535142e-05,
         2.85890129e-06,  9.50330192e-01, -2.66026576e-01,
         4.70308998e-03, -3.1

In [37]:
# Coefficients

λM_6 = np.exp(est_6.x[0])
λF_6 = np.exp(est_6.x[1])
ηM_6 = np.exp(est_6.x[2])
ηF_6 = np.exp(est_6.x[3])
μM_6 = est_6.x[4]
σM_6 = np.exp(est_6.x[5])
μF_6 = est_6.x[6]
σF_6 = np.exp(est_6.x[7])
d_6 = np.exp(est_6.x[8])
p_6 = np.exp(est_6.x[9])/(1+np.exp(est_6.x[9]))

print(λM_6, λF_6, ηM_6, ηF_6, μM_6, σM_6, μF_6, σF_6, d_6, p_6)

0.18000000000000002 0.28 0.0029982478582973393 0.0077002653651407835 3.896789177351564 0.9727407613526795 3.530924597863187 2.3989681825749334 5.5798376020333285 0.457723957946928


#### Estimation 3, Working!

In [57]:
def loglik_3( params: list ):
    """
    Calculates log likelihood with prejudice and productivity differences 
    
    Estimation 3 (η and h not gender specific)
    
    Parameters to estimate: 
        λ 
        η
        μM
        σM
        μF
        σF
        d
        p
    """
    
    λM = np.exp(params[0]) #same lambda
    λF = np.exp(params[0]) #same lambda
    η = np.exp(params[1]) #same eta
    μM = params[2]
    σM = np.exp(params[3])
    μF = params[4]
    σF = np.exp(params[5])
    d = np.exp(params[6])
    p = np.exp(params[7])/(1+np.exp(params[7]))
    
    # Men's equations
    λM = lambdaM(h, wstarM, α, μM, σM)
    
    a = M['dur'].count() * np.log(h/(h+η))
    b = MU['dur'].count() * np.log(η)
    c = - h * np.sum(MU.values[:,0])
    e = np.sum( np.log( (1/α) * dens_accepted(ME['wage'], α, μM, σM, wstarM) ) )
    
    
    # Women's equations
    λF = lambdaF(h, wstarF, α, μM, σM, p)
    
    f = F['dur'].count() * np.log(h/(h+η))
    g = FU['dur'].count() * np.log(η)
    i = - h * np.sum(FU.values[:,0])
    
    y = ((1-p)/α) * dens_accepted(FE['wage'], α, μF, σF, wstarF)
    z = (p/α) * dens_accepted_prej(FE['wage'], α, μF, σF, wstarF, d)

    j = np.sum( np.log( y + z ) )
    
    
    return a + b + c + e + f + g + i + j

In [58]:
# Check on log-likelihood

param3 = [λ, η, μ_M, σ_M, μ_F, σ_F, d, p]

b3_0 = np.log(param3[0])
b3_1 = np.log(param3[1])
b3_2 = param3[2]
b3_3 = np.log(param3[3])
b3_4 = param3[4]
b3_5 = np.log(param3[5])
b3_6 = np.log(param3[6])
b3_7 = np.log(1)

init3 = [b3_0, b3_1, b3_2, b3_3, b3_4, b3_5, b3_6, b3_7]

print(loglik_3(init3))

-5949.722619171719


In [59]:
est_3 = minimize(loglik_3, init3)

est_3

/Users/megan.mccoy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:36: RuntimeWarning: divide by zero encountered in log
/Users/megan.mccoy/anaconda3/lib/python3.6/site-packages/scipy/optimize/optimize.py:663: RuntimeWarning: invalid value encountered in double_scalars
  grad[k] = (f(*((xk + d,) + args)) - f0) / d[k]
/Users/megan.mccoy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:49: RuntimeWarning: divide by zero encountered in log
/Users/megan.mccoy/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1818: RuntimeWarning: overflow encountered in double_scalars
  x = np.asarray((x - loc)/scale, dtype=dtyp)
/Users/megan.mccoy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: divide by zero encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.
/Users/megan.mccoy/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1652: RuntimeWarning: divide by zero encoun

      fun: -14522.44804267152
 hess_inv: array([[ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  1.00000011e+00, -5.16200239e-05,
         1.29882541e-05, -6.61895835e-06,  1.97273741e-05,
         7.86477526e-05,  2.91649978e-07],
       [ 0.00000000e+00, -5.16200239e-05,  9.50330206e-01,
        -2.66026569e-01,  4.70308833e-03, -3.18816632e-03,
        -4.22641267e-02, -4.14508025e-03],
       [ 0.00000000e+00,  1.29882541e-05, -2.66026569e-01,
         7.50728792e-02,  4.59473896e-03, -5.81107175e-03,
        -5.44551097e-03, -1.15200661e-02],
       [ 0.00000000e+00, -6.61895835e-06,  4.70308833e-03,
         4.59473896e-03,  9.99824665e-01, -2.29876923e-02,
         2.29518829e-03,  2.14289380e-03],
       [ 0.00000000e+00,  1.97273741e-05, -3.18816632e-03,
        -5.81107175e-03, -2.29876923e-02,  9.94055426e-02,
         2.94047103e-01,  8.64042583

In [60]:
# Coefficients

λ_3 = np.exp(est_3.x[0])
η_3 = np.exp(est_3.x[1])
μM_3 = est_3.x[2]
σM_3 = np.exp(est_3.x[3])
μF_3 = est_3.x[4]
σF_3 = np.exp(est_3.x[5])
d_3 = np.exp(est_3.x[6])
p_3 = np.exp(est_3.x[7])/(1+np.exp(est_3.x[7]))

print(λ_3, η_3, μM_3, σM_3, μF_3, σF_3, d_3, p_3)

0.22 0.004996905021186945 3.896789204378965 0.9727407417765819 3.530924547840527 2.3989680828918742 5.579837042222459 0.45772395695445


#### Estimation 5, Working!

In [53]:
def loglik_5( params: list ):
    """
    Calculates log likelihood with prejudice, no productivity differences. 
    
    Estimation 5 (ηM, ηF, hM, hF)
    
    Parameters to estimate: 
        λM # see fxn elsewhere
        λF # see fxn elsewhere
        μ
        σ
        p
        d
    """

    λM = np.exp(params[0])
    λF = np.exp(params[1])
    ηM = np.exp(params[2])
    ηF = np.exp(params[3])
    μ = params[4]
    σ = np.exp(params[5])
    d = np.exp(params[6])
    p = np.exp(params[7])/(1+np.exp(params[7]))
    
    # Men's equations 
    λM = lambdaM(hM, wstarM, α, μ, σ)
    
    a = M['dur'].count() * np.log(hM/(hM+ηM))
    b = MU['dur'].count() * np.log(ηM)
    c = - hM * np.sum(MU.values[:,0])
    e = np.sum( np.log( (1/α) * dens_accepted(ME['wage'], α, μ, σ, wstarM) ) )
    
    
    # Women's equations
    λF = lambdaF(hF, wstarF, α, μ, σ, p)
    
    f = F['dur'].count() * np.log(hF/(hF+ηF))
    g = FU['dur'].count() * np.log(ηF)
    h = - hF * np.sum(FU.values[:,0])
    
    y = ((1-p)/α) * dens_accepted(FE['wage'], α, μ, σ, wstarF)
    z = (p/α) * dens_accepted_prej(FE['wage'], α, μ, σ, wstarF, d)

    i = np.sum( np.log( y + z ) )
    
    
    return a + b + c + e + f + g + h + i

In [54]:
# Check on log-likelihood

param5 = [λ_M, λ_F, η_M, η_F, μ, σ, d, p]

b5_0 = np.log(param6[0])
b5_1 = np.log(param6[1])
b5_2 = np.log(param6[2])
b5_3 = np.log(param6[3])
b5_4 = param6[4]
b5_5 = np.log(param6[5])
b5_6 = np.log(param6[6])
b5_7 = np.log(1)

init5 = [b5_0, b5_1, b5_2, b5_3, b5_4, b5_5, b5_6, b5_7]

print(loglik_5(init5))

-11513.481093249044


In [55]:
est_5 = minimize(loglik_5, init5)

est_5

/Users/megan.mccoy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:31: RuntimeWarning: divide by zero encountered in log
/Users/megan.mccoy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: RuntimeWarning: divide by zero encountered in log
/Users/megan.mccoy/anaconda3/lib/python3.6/site-packages/scipy/optimize/optimize.py:663: RuntimeWarning: invalid value encountered in double_scalars
  grad[k] = (f(*((xk + d,) + args)) - f0) / d[k]
/Users/megan.mccoy/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1818: RuntimeWarning: overflow encountered in double_scalars
  x = np.asarray((x - loc)/scale, dtype=dtyp)
/Users/megan.mccoy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: divide by zero encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.
/Users/megan.mccoy/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1652: RuntimeWarning: divide by zero encoun

      fun: -14500.787761901043
 hess_inv: array([[ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  1.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  1.00000001e+00,
        -8.17222959e-10, -7.91351077e-06,  2.73222802e-06,
         6.26952406e-06,  4.59716640e-07],
       [ 0.00000000e+00,  0.00000000e+00, -8.17222959e-10,
         1.00000000e+00,  4.65105487e-07, -1.66820453e-07,
        -3.68635791e-07, -2.69184024e-08],
       [ 0.00000000e+00,  0.00000000e+00, -7.91351077e-06,
         4.65105487e-07,  9.54375414e-01, -2.17021500e-01,
         3.04249310e-02,  6.41201910e-03],
       [ 0.00000000e+00,  0.00000000e+00,  2.73222802e-06,
        -1.66820453e-07, -2.17021500e-01,  7.46792903e-02,
         1.47351131e-01,  2.8771131

In [56]:
# Coefficients

λM_5 = np.exp(est_5.x[0])
λF_5 = np.exp(est_5.x[1])
ηM_5 = np.exp(est_5.x[2])
ηF_5 = np.exp(est_5.x[3])
μ_5 = est_5.x[4]
σ_5 = np.exp(est_5.x[5])
d_5 = np.exp(est_5.x[6])
p_5 = np.exp(est_5.x[7])/(1+np.exp(est_5.x[7]))

print(λM_5, λF_5, ηM_5, ηF_5, μ_5, σ_5, d_5, p_5)

0.18000000000000002 0.28 0.002999644963371521 0.0077000537625960454 3.7486253410117842 1.4339576332057262 2.810552051335424 0.49152977167263917


#### Estimation 2 (very small d, but plausable)

In [68]:
def loglik_2( params: list ):
    """
    Calculates log likelihood with prejudice, no productivity differences. 
    
    Estimation 2 (η and h not gender specific)
    
    Parameters to estimate: 
        λ # see fxn elsewhere
        μ
        σ
        p
        d
    """
    
    λM = np.exp(params[0])
    λF = np.exp(params[0])
    η = np.exp(params[1])
    μ = params[2]
    σ = np.exp(params[3])
    d = np.exp(params[4])
    p = np.exp(params[5])/(1+np.exp(params[5]))

    
    # Men's equations 
    λM = lambdaM(h, wstarM, α, μ, σ)
    
    a = M['dur'].count() * np.log(h/(h+η))
    b = MU['dur'].count() * np.log(η)
    c = - h * np.sum(MU.values[:,0])
    e = np.sum( np.log( (1/α) * dens_accepted(ME['wage'], α, μ, σ, wstarM) ) )
    
    
    # Women's equations
    λF = lambdaF(h, wstarF, α, μ, σ, p)
    
    f = F['dur'].count() * np.log(h/(h+η))
    g = FU['dur'].count() * np.log(η)
    i = - h * np.sum(FU.values[:,0])
    
    y = ((1-p)/α) * dens_accepted(FE['wage'], α, μ, σ, wstarF)
    z = (p/α) * dens_accepted_prej(FE['wage'], α, μ, σ, wstarF, d)

    j = np.sum( np.log( y + z ) )
    
    
    return a + b + c + e + f + g + i + j

In [73]:
# Check on log-likelihood

param2 = [λ, η, μ, σ, d, p]

b2_0 = np.log(param2[0])
b2_1 = np.log(param2[1])
b2_2 = param2[2]
b2_3 = np.log(param2[3])
b2_4 = np.log(param2[4])
b2_5 = np.log(1)

init2 = [b2_0, b2_1, b2_2, b2_3, b2_4, b2_5]

print(loglik_2(init2))

-6487.137606363093


In [74]:
est_2 = minimize(loglik_2, init2)

est_2

/Users/megan.mccoy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:30: RuntimeWarning: divide by zero encountered in log
/Users/megan.mccoy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:43: RuntimeWarning: divide by zero encountered in log
/Users/megan.mccoy/anaconda3/lib/python3.6/site-packages/scipy/optimize/optimize.py:663: RuntimeWarning: invalid value encountered in double_scalars
  grad[k] = (f(*((xk + d,) + args)) - f0) / d[k]
/Users/megan.mccoy/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1818: RuntimeWarning: overflow encountered in double_scalars
  x = np.asarray((x - loc)/scale, dtype=dtyp)
/Users/megan.mccoy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: divide by zero encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.
/Users/megan.mccoy/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1652: RuntimeWarning: divide by zero encoun

      fun: -15858.733214820882
 hess_inv: array([[ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  1.00000078e+00, -3.51718010e-04,
         8.30301637e-05,  1.97572447e-04,  2.27739450e-05],
       [ 0.00000000e+00, -3.51718010e-04,  1.01872132e+00,
        -2.33539975e-01, -3.67287978e-01, -7.40088282e-02],
       [ 0.00000000e+00,  8.30301637e-05, -2.33539975e-01,
         5.36828040e-02,  8.60946341e-02,  9.84866862e-03],
       [ 0.00000000e+00,  1.97572447e-04, -3.67287978e-01,
         8.60946341e-02,  1.83326582e-01, -1.87602824e-01],
       [ 0.00000000e+00,  2.27739450e-05, -7.40088282e-02,
         9.84866862e-03, -1.87602824e-01,  9.57466755e-01]])
      jac: array([ 0.00000000e+00,  1.83374023e+00,  3.95988159e+02,  6.22888110e+03,
       -9.91027832e+00, -7.25891113e+00])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 852
      nit: 2
     njev: 1

In [75]:
# Coefficients

λ_2 = np.exp(est_2.x[0])
η_2 = np.exp(est_2.x[1])
μ_2 = est_2.x[2]
σ_2 = np.exp(est_2.x[3])
d_2 = np.exp(est_2.x[4])
p_2 = np.exp(est_2.x[5])/(1+np.exp(est_2.x[5]))

print(λ_2, η_2, μ_2, σ_2, d_2, p_2)

0.22 0.004983119938082697 3.9852795832274084 1.0903744533318382 0.17630260835128692 0.2904171910575877


#### Estimation 4, Working!

In [80]:
def loglik_4( params: list ):
    """
    Calculates log likelihood with productivity differences, no prejudice
    
    Estimation 4 (ηM, ηF, hM, hF)
    
    Parameters to estimate: 
        λM # see fxn elsewhere
        λF # see fxn elsewhere
        μM
        σM
        μF
        σF
    """

    λM = np.exp(params[0])
    λF = np.exp(params[1])
    ηM = np.exp(params[2])
    ηF = np.exp(params[3])
    μM = params[4]
    σM = np.exp(params[5])
    μF = params[6]
    σF = np.exp(params[6])
    p = 0
    
    # Men's equations 
    λM = lambdaM(hM, wstarM, α, μM, σM)
    
    a = M['dur'].count() * np.log(hM/(hM+ηM))
    b = MU['dur'].count() * np.log(ηM)
    c = - hM * np.sum(MU.values[:,0])
    e = np.sum( np.log( (1/α) * dens_accepted(ME['wage'], α, μM, σM, wstarM) ) )
    
    
    # Women's equations
    λF = lambdaF(hF, wstarF, α, μF, σF, p)
    
    f = F['dur'].count() * np.log(hF/(hF+ηF))
    g = FU['dur'].count() * np.log(ηF)
    h = - hF * np.sum(FU.values[:,0])
    
    y = (1/α) * dens_accepted(FE['wage'], α, μF, σF, wstarF)

    i = np.sum( np.log( y ) )
    
    
    return a + b + c + e + f + g + h + i

In [81]:
# Check on log-likelihood

param4 = [λ_M, λ_F, η_M, η_F, μ_M, σ_M, μ_F, σ_F]

b4_0 = np.log(param4[0])
b4_1 = np.log(param4[1])
b4_2 = np.log(param4[2])
b4_3 = np.log(param4[3])
b4_4 = param4[4]
b4_5 = np.log(param4[5])
b4_6 = param4[6]
b4_7 = np.log(param4[7])

init4 = [b4_0, b4_1, b4_2, b4_3, b4_4, b4_5, b4_6, b4_7]

print(loglik_4(init4))

-11217.133954132982


In [82]:
est_4 = minimize(loglik_4, init4)

est_4

/Users/megan.mccoy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:32: RuntimeWarning: divide by zero encountered in log
/Users/megan.mccoy/anaconda3/lib/python3.6/site-packages/scipy/optimize/optimize.py:663: RuntimeWarning: invalid value encountered in double_scalars
  grad[k] = (f(*((xk + d,) + args)) - f0) / d[k]
/Users/megan.mccoy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: RuntimeWarning: divide by zero encountered in log
/Users/megan.mccoy/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1818: RuntimeWarning: overflow encountered in double_scalars
  x = np.asarray((x - loc)/scale, dtype=dtyp)
/Users/megan.mccoy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: divide by zero encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.
/Users/megan.mccoy/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1652: RuntimeWarning: divide by zero encoun

      fun: -13285.255387569408
 hess_inv: array([[ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  1.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  1.00000003e+00,
        -1.86764823e-09, -1.58008390e-05,  4.17127214e-06,
        -8.04299628e-06,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00, -1.86764823e-09,
         1.00000000e+00,  9.33723474e-07, -2.41876863e-07,
         4.74686980e-07,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00, -1.58008390e-05,
         9.33723474e-07,  9.39590087e-01, -2.54337763e-01,
         4.37746378e-03,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  4.17127214e-06,
        -2.41876863e-07, -2.54337763e-01,  6.88818081e-02,
         2.74670760e-04,  0.0000000

In [83]:
# Coefficients

λM_4 = np.exp(est_4.x[0])
λF_4 = np.exp(est_4.x[1])
ηM_4 = np.exp(est_4.x[2])
ηF_4 = np.exp(est_4.x[3])
μM_4 = est_4.x[4]
σM_4 = np.exp(est_4.x[5])
μF_4 = est_4.x[6]
σF_4 = np.exp(est_4.x[7])

print(λM_4, λF_4, ηM_4, ηF_4, μM_4, σM_4, μF_4, σF_4)

0.18000000000000002 0.28 0.002999790031996455 0.007700031794285429 3.593775162780644 0.8082804191357728 3.4712454171546696 0.423


#### Estimation 1, Working!

In [84]:
def loglik_1( params: list ):
    """
    Calculates log likelihood with productivity differences, no prejudice
    
    Estimation 1 (η and h not gender specific)
    
    Parameters to estimate: 
        λ
        μM
        σM
        μF
        σF
    """

    λM = np.exp(params[0]) #same lambda
    λF = np.exp(params[0]) #same lambda
    η = np.exp(params[1]) #same eta
    μM = params[2]
    σM = np.exp(params[3])
    μF = params[4]
    σF = np.exp(params[5])
    p = 0
    
    # Men's equations 
    λM = lambdaM(h, wstarM, α, μM, σM)
    
    a = M['dur'].count() * np.log(h/(h+η))
    b = MU['dur'].count() * np.log(η)
    c = - h * np.sum(MU.values[:,0])
    e = np.sum( np.log( (1/α) * dens_accepted(ME['wage'], α, μM, σM, wstarM) ) )
    
    
    # Women's equations
    λF = lambdaF(h, wstarF, α, μF, σF, p)
    
    f = F['dur'].count() * np.log(h/(h+η))
    g = FU['dur'].count() * np.log(η)
    i = - h * np.sum(FU.values[:,0])
    
    y = (1/α) * dens_accepted(FE['wage'], α, μF, σF, wstarF)

    j = np.sum( np.log( y ) )
    
    
    return a + b + c + e + f + g + i + j

In [85]:
# Check on log-likelihood

param1 = [λ, η, μ_M, σ_M, μ_F, σ_F]

b1_0 = np.log(param1[0])
b1_1 = np.log(param1[1])
b1_2 = param1[2]
b1_3 = np.log(param1[3])
b1_4 = param1[4]
b1_5 = np.log(param1[5])

init1 = [b1_0, b1_1, b1_2, b1_3, b1_4, b1_5]

print(loglik_1(init1))

-15716.39689619685


In [86]:
est_1 = minimize(loglik_1, init1)

est_1

/Users/megan.mccoy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:42: RuntimeWarning: divide by zero encountered in log
/Users/megan.mccoy/anaconda3/lib/python3.6/site-packages/scipy/optimize/optimize.py:663: RuntimeWarning: invalid value encountered in double_scalars
  grad[k] = (f(*((xk + d,) + args)) - f0) / d[k]
/Users/megan.mccoy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:30: RuntimeWarning: divide by zero encountered in log
/Users/megan.mccoy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: divide by zero encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.
/Users/megan.mccoy/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1652: RuntimeWarning: divide by zero encountered in true_divide
  x = np.asarray((x - loc)/scale, dtype=dtyp)
/Users/megan.mccoy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in doubl

      fun: -19351.430213370506
 hess_inv: array([[ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  1.00001173e+00, -5.63423683e-03,
         2.50624001e-03, -6.91637888e-03,  2.29218069e-03],
       [ 0.00000000e+00, -5.63423683e-03,  3.52593855e+00,
        -1.56791893e+00,  3.38360485e+00, -9.89720589e-01],
       [ 0.00000000e+00,  2.50624001e-03, -1.56791893e+00,
         6.97390700e-01, -1.49196393e+00,  4.33970384e-01],
       [ 0.00000000e+00, -6.91637888e-03,  3.38360485e+00,
        -1.49196393e+00,  4.89132962e+00, -1.73784639e+00],
       [ 0.00000000e+00,  2.29218069e-03, -9.89720589e-01,
         4.33970384e-01, -1.73784639e+00,  6.55721445e-01]])
      jac: array([ 0.00000000e+00,  1.76904297e+00, -2.48376978e+03, -2.30539600e+03,
        2.42332275e+02,  2.89076294e+03])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 908
      nit: 2
     njev: 1

In [87]:
# Coefficients

λ_1 = np.exp(est_1.x[0])
η_1 = np.exp(est_1.x[1])
μM_1 = est_1.x[2]
σM_1 = np.exp(est_1.x[3])
μF_1 = est_1.x[4]
σF_1 = np.exp(est_1.x[5])

print(λ_1, η_1, μM_1, σM_1, μF_1, σF_1)

0.22 0.004990775379360802 4.321047691447253 0.5231501431192599 5.017328401110969 0.7873760747884067
